In [ ]:
!./interactive_train-jasmine.sh

In [16]:
from transformers import AutoModelForCausalLM, pipeline

#model = AutoModelForCausalLM.from_pretrained("Output_Dir/Testing_1/gpt2")

generator = pipeline('text-generation', model="Output_Dir/Testing_1/gpt2", tokenizer = "Output_Dir/Testing_1/gpt2", use_fast = False)# device_map='auto')

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM

path = "Output_Dir/No_Expl_LLama_Chat_7b_1/Llama-2-7b-chat-hf/checkpoint-116"

tokenizer = LlamaTokenizer.from_pretrained(path)
model = LlamaForCausalLM.from_pretrained(path)

In [3]:
from transformers import AutoModelForCausalLM, pipeline

#model = AutoModelForCausalLM.from_pretrained("Output_Dir/Testing_1/gpt2")

generator = pipeline('text-generation', model=model, tokenizer = tokenizer, device = 0) #use_fast = False)# device_map='auto')

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [ ]:
prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "
inp = "2. \tFucking THIS.  Too many retards on this sub demand that Gab stand purely on principle when its their future on the line. I can guarantee nobody here would have the balls to do it either."
prompt = prompt+inp+"\n"
res = generator(prompt)
print(res[0]['generated_text'])

In [4]:
import pandas as pd

test_file = 'Datasets/Toxicity_All_Platform_3K/test/test_All_Platform_3K.csv'
df_test = pd.read_csv(test_file)

def process(text):
    #prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "
    prompt = "Rewrite the following toxic input into non-toxic version. You must preserve the original meaning as much as possible.\n Input: "
    text = prompt+text+"\n"
    return text

df_test['input'] = df_test['toxic'].apply(process)
    


In [8]:
df_test['input'][1]

"Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: how the hell did we manage to fucking disappoint the goddamn overrated mike posner band like this? i'm so fucking ashamed and utterly humiliated. https://t.co/XQdWqEBVau\n"

In [ ]:
gen_list = generator(df_test['input'].tolist(), max_new_tokens=256) ### 512

In [ ]:
print(gen_list[0][0]['generated_text'])

In [ ]:
def get_detox_expl(text):
    detox_list = text.split('Detoxification:')
    if(len(detox_list)==1):
        return '',''
    detox = detox_list[1].strip()
    expl = detox_list[0].split('Explanation:')[-1].strip()
    return detox, expl

In [ ]:
predictions = []
pred_explanations = []

for i,gen in enumerate(gen_list):
    #print(gen[0])
    #try:
    pred, expl = get_detox_expl(gen[0]['generated_text'])
    predictions.append(pred)
    pred_explanations.append(expl)
    '''except:
        print(i)
        print(gen[0]['generated_text'])
        print("******")'''

sources = df_test['toxic'].tolist()
references = df_test['non_toxic'].tolist()
ref_explanations = df_test['explanation'].tolist()


In [ ]:
import os
import json

output_prediction_file = os.path.join(path, "Xplatform_generated_predictions.jsonl")

with open(output_prediction_file, "w") as writer:             
    for src, ref, pred, pred_expl, ref_expl in zip(sources, references, predictions, pred_explanations, ref_explanations):
        output = {
            'source': src,
            'reference': ref,
            'prediction': pred,
            'predicted_explanation': pred_expl,
            'ChatGPT_explanation': ref_expl,
        }
        json.dump(output, writer)
        #writer.write("REFERENCES: "+str(ref))
        #writer.write("\nPREDICTION: "+str(pred))
        writer.write("\n")

In [ ]:
import pandas as pd

df_predicted = pd.read_json(output_prediction_file, lines=True)

df_predicted['index'] = df_predicted.index

df_predicted.to_excel(f'{path}/Xplatform_generated_predictions.xlsx', index=False)

In [1]:
cd "Output_Dir/No_Expl_LLama_Chat_7b_1/Llama-2-7b-chat-hf/checkpoint-116"

/lustre07/scratch/tawkat/FT_Abusive/Output_Dir/LLama_Chat_7b_2/Llama-2-7b-chat-hf/checkpoint-116


In [2]:
!python ./zero_to_fp32.py . pytorch_model.bin

[2023-08-04 16:10:46,446] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda114.so
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/tmp/tawkat')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('-10')}
  warn(msg)
/home/tawkat/ENV_1/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py